In [3]:
import pickle as pkl
import pandas as pd
import numpy as np
import pytorch_nndct
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_nndct import Inspector
batch_size = 1
# load data
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
    
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
    
        return normalized_sample, label

train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


[VAIQ_NOTE]: Loading NNDCT kernels...


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [20]:
"""class ModifiedToyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='nearest')  # set mode = "nearest"
        self.conv = torch.nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu = torch.nn.ReLU()
        self.adaptive_avg_pool2d = torch.nn.AdaptiveAvgPool2d(output_size=1)
        
    def forward(self, x):
        x = self.upsample(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.adaptive_avg_pool2d(x)
        x = x.permute(0, 2, 3, 1) # insert a permute to cancel out the permute inserted by quantizer.
        x = x.reshape(x.size(0), -1)
        return x

# initialize model
num_classes = 2 #BPSK / QPSK
model = CNN1D(num_classes)
"""
import torch
import torch.nn as nn

class CNN2D(nn.Module):
    def __init__(self, num_classes):
        super(CNN2D, self).__init__()
        
        in_channels = 2
        # For the 2D conversion, we're assuming height=1 for the input
        height = 1
        width = 128  # Original length dimension
        
        # Upsample (adjusted for 2D)
        self.upsample = nn.Upsample(scale_factor=(1, 2), mode='bilinear', align_corners=False)  # height stays the same, width is doubled
        
        # Define 2D convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(1, 3), padding=(0, 1))
        self.relu1 = nn.ReLU()
        #self.maxpool1 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool1 = nn.AdaptiveAvgPool2d((1,64))
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1))
        self.relu2 = nn.ReLU()
        #self.maxpool2 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool2 = nn.AdaptiveAvgPool2d((1,32))
        
        # Adaptive pooling (2D)
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        # Define the fully connected layers (adjusted for adaptive pooling output)
        self.fc1 = nn.Linear(128, 256)  # The input features after pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # Adjust input shape for 2D processing
        x = x.unsqueeze(-2)  # Insert a height dimension of size 1
        
        # Upsample
        x = self.upsample(x)
        
        # 2D convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.maxpool1(x)
        x = self.adaptive_pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        #x = self.maxpool2(x)
        x = self.adaptive_pool2(x)
        
        # Adaptive average pooling
        x = self.adaptive_avg_pool2d(x)
        
        # Flatten (before fully connected layers)
        x = x.reshape(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x


model = CNN2D(2)
print(model) #model info

CNN2D(
  (upsample): Upsample(scale_factor=(1.0, 2.0), mode=bilinear)
  (conv1): Conv2d(2, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu1): ReLU()
  (adaptive_pool1): AdaptiveAvgPool2d(output_size=(1, 64))
  (conv2): Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu2): ReLU()
  (adaptive_pool2): AdaptiveAvgPool2d(output_size=(1, 32))
  (adaptive_avg_pool2d): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


In [21]:
# Specify a target name or fingerprint you want to deploy on
#target = "DPUCAHX8L_ISA0_SP"
target = "DPUCVDX8G_ISA3_C32B3"
#DPUCVDX8G
# Initialize inspector with target
inspector = Inspector(target)
# Note: visualization of inspection results relies on the dot engine.If you don't install dot successfully, set 'image_format = None' when inspecting.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN2D(2)

#dummy_data, labels = train_loader
#data_dummy = []
for data, labels in train_loader:
    data_dummy = data
    break
print(type(data_dummy))
print(data_dummy.shape)
dummy_input = torch.randn(1, 2, 128)
print(dummy_input)

inspector.inspect(model, (data_dummy.clone().detach().requires_grad_(True)), device=device, output_dir="inspect", image_format=None) 


[VAIQ_NOTE]: Inspector is on.
<class 'torch.Tensor'>
torch.Size([1, 2, 128])
tensor([[[-3.0413e-01, -1.1877e+00, -7.7897e-01, -5.9376e-01,  2.6584e-01,
           3.0836e-01, -1.2525e+00, -2.2555e-01, -4.9644e-02,  4.1900e-01,
           1.4311e+00,  3.7387e-01, -9.2353e-01,  7.0805e-01, -2.9606e+00,
          -3.7607e-01, -1.5334e+00,  1.2445e+00,  1.4622e+00,  3.1517e-01,
          -1.1061e+00,  6.4145e-01,  1.1340e+00,  3.5031e-02,  4.2173e-01,
          -7.1159e-01,  9.8216e-02, -2.5417e+00,  7.2321e-01,  3.1083e-01,
           1.5646e+00,  1.3674e+00,  6.6961e-01,  9.6399e-01,  1.3892e+00,
           7.2034e-01, -8.0976e-01, -8.6055e-01, -1.9259e-01,  1.9071e-02,
          -2.2167e-01, -7.3138e-03,  1.5966e-01,  1.2113e+00, -9.4370e-01,
           6.6354e-01, -1.8099e-01,  4.1636e-01, -1.0002e+00, -1.3792e+00,
           2.8513e-01,  4.5752e-02, -1.0645e+00, -1.3079e+00, -2.3473e-01,
          -5.7306e-01,  1.0545e+00, -5.8274e-01,  7.2737e-01,  1.6759e+00,
           7.2980e-01,

/tmp/ipykernel_152540/822561179.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_152540/822561179.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_152540/822561179.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
██████████████████████████████████████████████████| 16/16 [00:00<00:00, 2321.38it/s, OpInfo: n


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(inspect/CNN2D.py)

[VAIQ_WARN]: CNN2D::507 is not tensor.

[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Linear[fc1]/ret.19, op type:nndct_dense, output shape: [1, 256]
node name:CNN2D::CNN2D/ReLU[relu3]/ret.21, op type:nndct_relu, output shape: [1, 256]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Conv2d[conv2]/ret.11, op type:nndct_conv2d, output shape: [1, 1, 64, 128]
node name:CNN2D::CNN2D/ReLU[relu2]/ret.13, op type:nndct_relu, output shape: [1, 1, 64, 128]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Conv2d[conv1]/ret.7, op type:nndct_conv2d, output shape: [1, 1, 256, 64]
node name:CNN2D::CNN2D/ReLU[relu1]/ret.9, op type:nndct_relu, output shape: [1, 1, 256, 64]



[VAIQ_NOTE]: Find subgraph for reshape_fix_1:
node name:CNN2D::CNN2D/ret.3, op type:nndct_reshape, output shape: [1, 2, 1, 128]



[VAIQ_NOTE]


I20240227 10:14:59.096123 152540 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240227 10:14:59.096146 152540 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240227 10:14:59.096150 152540 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCVDX8G_ISA3_C32B3
I20240227 10:14:59.096243 152540 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_dense_nndct_relu_8ZfdOuePFXBRi2pJ, with op num: 9
I20240227 10:14:59.096248 152540 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240227 10:14:59.100780 152540 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20240227 10:14:59.100800 152540 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240227 10:14:59.103451 152540 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240227 10:14:59.103459 152540 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240227 10:14:59.103462 152

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


learning_rate = 0.001
num_epochs = 3

# loss function and optimizer definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training
for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    
    for data, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    
    # print loss
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')

# evaluate the model on test data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # predicted and actual labels
        for i in range(len(labels)):
            print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

/tmp/ipykernel_152540/822561179.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_152540/822561179.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_152540/822561179.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


Epoch [1/3] Loss: 0.1151
Epoch [2/3] Loss: 0.0808
Epoch [3/3] Loss: 0.0258
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1,

In [ ]:
print(total)

In [ ]:
torch.save(model, "trainedModel")